# 1. Description of the project

In this project, a RAG system is implemented and used in combination with LettuceDetect.

# 2. Setup

1. **Install these packages:**

In [54]:
%pip install -qq langchain langchain-unstructured langchain-chroma langchain-openai unstructured langchain-community unstructured[pdf] dotenv lettucedetect gradio

Note: you may need to restart the kernel to use updated packages.


2. **Import the necessary modules**

In [55]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_unstructured import UnstructuredLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_chroma.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata
from langchain_core.documents.base import Document
import os
from dotenv import load_dotenv, find_dotenv
from lettucedetect.models.inference import HallucinationDetector
import gradio as gr

3. **Deploy an Azure OpenAI LLM resource and embedding resource**

    Use the following link: https://ai.azure.com/
4. **Save the details to the .env file:**
    ```bash
    echo AZURE_OPENAI_API_KEY=\"your-api-key-here\" >> .env
    echo AZURE_OPENAI_API_VERSION=\"your-version-here\" >> .env
    echo AZURE_OPENAI_ENDPOINT=\"your-endpoint-here\" >> .env
    echo GPT_MODEL=\"your-llm-model-here\" >> .env
    echo EMBEDDINGS_MODEL_NAME=\"your-embeddings-model-here\" >> .env
    echo EMBEDDINGS_DEPLOYMENT=\"your-embeddings-deployment-here\" >> .env
    ```

# 3. ChromaDB setup

## 3.1 The text splitter

The text splitter divides documents into manageable chunks to optimize downstream processing and retrieval in RAG workflows.

In [56]:

def text_splitter(data, debug = False):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=50,
        length_function=len,
    )
    if debug:
        print(f"Splitting {len(data)} documents into chunks...")
    chunks = text_splitter.split_documents(data)
    return chunks

## 3.2 The document loader

The document loader reads and parses files from the corpus directory into structured document objects for downstream processing.

In [57]:
def load_documents(corpus_dir = "./corpus/", debug = False):
    # Load all documents from the corpus directory
    loaded_docs = []
    if debug:
        print(f"Loading documents from {corpus_dir}...")
    for file in os.listdir(corpus_dir):
        if debug:
            print(f"Loading {file}...")
        loader = UnstructuredLoader(corpus_dir + file, mode = 'single')
        loaded_docs.extend(loader.load())

    # Filter complex metadata from loaded documents
    if debug:
        print("Filtering complex metadata...")
    filtered_docs = filter_complex_metadata(loaded_docs)

    return filtered_docs

## 3.3 The embedding client

The embedding client initializes and manages Azure OpenAI embeddings for converting text into vector representations.

In [58]:
def embeddings(debug = False):
    load_dotenv(find_dotenv())
    model = os.getenv('EMBEDDINGS_MODEL_NAME')
    api_key = os.getenv('AZURE_OPENAI_API_KEY')
    api_version = os.getenv("AZURE_OPENAI_API_VERSION")
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    azure_deployment = os.getenv("EMBEDDINGS_DEPLOYMENT")

    # Validate required environment variables
    if not all([model, api_key, api_version, azure_endpoint, azure_deployment]):
        raise ValueError(
            """
            Missing environment variables.
            Please load all the required environment variables in the .env file:
            EMBEDDINGS_MODEL_NAME, AZURE_OPENAI_API_KEY, AZURE_OPENAI_API_VERSION,
            AZURE_OPENAI_ENDPOINT, EMBEDDINGS_DEPLOYMENT
            """
        )
    
    # Initialize and return an Azure OpenAI embeddings client
    if debug:
        print(f"Initializing embeddings with model: {model}, deployment: {azure_deployment}")
    embeddings = AzureOpenAIEmbeddings(
        model = model,
        api_key = api_key,
        api_version = api_version,
        azure_endpoint = azure_endpoint,
        azure_deployment = azure_deployment,
    )
    return embeddings

## 3.4 The vector database

The vector database stores document embeddings for fast similarity search and retrieval. Built with Chroma, it enables efficient access to relevant document chunks in RAG workflows.

In [59]:
def create_database(document_list, debug = False):
    # Initialize the database from a given corpus of documents
    embedding_model = embeddings(debug = debug)

    # Create and return a Chroma vector store from the documents
    if debug:
        print(f"Creating vector database with {len(document_list)} documents...")
    return Chroma.from_documents(documents = document_list,
                                            embedding = embedding_model)

## 3.5 The retriever

The retriever fetches relevant document chunks from the vector database using embeddings to match user queries with semantically similar content for efficient retrieval in RAG workflows. It is not based on LLMs, but purely on a similarity search algorithm.

In [60]:
def retriever(corpus_dir = "./corpus/", debug = False):
    docs = load_documents(corpus_dir, debug = debug)
    chunks = text_splitter(docs, debug = debug)
    vectordb = create_database(chunks, debug = debug)
    if debug:
        print("Creating retriever from vector database...")
    retriever = vectordb.as_retriever()
    return retriever

## 3.6 Small test

In [61]:
test_retriever = retriever(corpus_dir = "./example_inputs/", debug = True)

Loading documents from ./example_inputs/...
Loading ford_f150_lightning_2024.txt...
Loading audi_a4_2024.txt...
Loading mercedes_cclass_2024.txt...
Loading tesla_model_s_2024.txt...
Loading toyota_camry_2024.txt...
Loading bmw_3series_2024.txt...
Loading jeep_wrangler_2024.txt...
Loading porsche_911_carrera_2024.txt...
Loading subaru_outback_2024.txt...
Loading honda_accord_2024.txt...
Filtering complex metadata...
Splitting 271 documents into chunks...
Initializing embeddings with model: text-embedding-3-small, deployment: text-embedding-3-small
Creating vector database with 271 documents...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Creating retriever from vector database...


In [62]:
docs = test_retriever.get_relevant_documents("Which car features a turbocharged 3.0-liter inline-six engine")
docs

INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


[Document(id='0ffcfc39-1795-4eab-afcc-b61ac8917e5c', metadata={'source': './example_inputs/honda_accord_2024.txt', 'filetype': 'text/plain', 'category': 'NarrativeText', 'filename': 'honda_accord_2024.txt', 'element_id': '56c4f36868129724874376f4e43bb081', 'file_directory': './example_inputs', 'last_modified': '2025-07-24T15:17:12', 'parent_id': '0749961253165664e3d63731c5667eee'}, page_content='For those requiring additional power, the available 2.0-liter turbocharged four-cylinder engine generates an impressive 252 horsepower and 273 lb-ft of torque. This engine features a sophisticated cooling system and advanced turbocharger technology that delivers smooth, linear power delivery throughout the rev range.'),
 Document(id='9ceaf92f-f070-4cdb-8867-981a361328e7', metadata={'parent_id': '0749961253165664e3d63731c5667eee', 'filetype': 'text/plain', 'file_directory': './example_inputs', 'element_id': '56c4f36868129724874376f4e43bb081', 'last_modified': '2025-07-24T15:17:12', 'category': '

# 4. The Hallucination detector

In [63]:
def detect_hallucinations(context, question, answer, debug = False):
    if debug:
        print(f"Predicting hallucination for question: {question}")

    # Initialize the hallucination detector with a transformer model
    detector = HallucinationDetector(
        method="transformer",
        model_path="KRLabsOrg/lettucedect-base-modernbert-en-v1"
    )

    # Predict hallucination using the detector
    result = detector.predict(context = context,
                              question = question,
                              answer = answer,
                              output_format = "spans")
    return result

# 5. The interface

In [64]:
def gradio_backend(uploaded_files, question, answer, debug = False):
    # Create a new corpus directory
    corpus_dir = "./corpus/"
    if os.path.exists(corpus_dir):
        for f in os.listdir(corpus_dir):
            os.remove(os.path.join(corpus_dir, f))
    os.makedirs(corpus_dir, exist_ok=True)

    # Save the uploaded files to corpus directory
    for file in uploaded_files:
        filename = os.path.basename(file.name)
        src = open(file.name, "rb")
        dst = open(os.path.join(corpus_dir, filename), "wb")
        dst.write(src.read())

    # Should take a list of strings as context
    if debug:
        print(f"Detecting hallucination for question: {question} with answer: {answer}")
    
    # Initialize the retriever with the corpus directory
    doc_retriever = retriever(corpus_dir = corpus_dir, debug = debug)

    # Retrieve relevant documents from the corpus
    if debug:
        print(f"Retrieving relevant documents for question: {question}")
    retrieved_docs_question = doc_retriever.get_relevant_documents(question)
    retrieved_docs_answer = doc_retriever.get_relevant_documents(answer)
    retrieved_docs = retrieved_docs_question + retrieved_docs_answer

    # Predict hallucination using the predict_hallucination function
    detected_hallucination = detect_hallucinations(
        context = retrieved_docs,
        question = question,
        answer = answer,
        debug = debug
    )

    # Check if an hallucination was detected
    if detected_hallucination:
        hallucination_was_found = "Hallucinations detected"
    else:
        hallucination_was_found = "No hallucinations found"

    # Create an output string based on the result
    hallucination_str = ""
    for hallucination in detected_hallucination:
        hallucination_str += f"""
           '{hallucination['text']}' - Confidence = {hallucination['confidence']}\n
        """

    return hallucination_was_found, hallucination_str


In [ ]:
rag_application = gr.Interface(
    fn = gradio_backend,
    allow_flagging = "never",
    inputs = [
        # Drag and drop files, returns a list of file paths
        gr.File(label = "Upload PDF/txt files",
                file_count = 'multiple',
                file_types = ['.pdf', '.txt']),
        gr.Textbox(label = "Prompt",
                   placeholder = "Type your question here..."),
        gr.Textbox(label = "Answer",
                   lines = 3,
                   placeholder = "type the answer here..."),
    ],
    outputs = [
        gr.Textbox(label = "Status"),
        gr.Textbox(label = "Detected Hallucinations",
                   lines = 5)
    ],
    title = "RAG system with Hallucination Detection",
    description = """
        Upload a collection of pdf or txt files provide a prompt and a response.
        The backend will try to detect hallucinations in the response based on the provided context.
    """
)

rag_application.launch()

/home/pau/python/lettuce_detect/.venv/lib/python3.12/site-packages/gradio/interface.py:425: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(
INFO: HTTP Request: GET http://127.0.0.1:7861/gradio_api/startup-events "HTTP/1.1 200 OK"
INFO: HTTP Request: HEAD http://127.0.0.1:7861/ "HTTP/1.1 200 OK"


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


INFO: HTTP Request: GET https://api.gradio.app/pkg-version "HTTP/1.1 200 OK"


# 6. Tests

After completing the setup, several tests were carried in order to ensure the optimal performance of the system. Each one reflects a state of the project, and justifies the adjustements that were made in the code in order to improve the performance.

### Test 1

This test was done before the answer was included in the input for retrieving relevant chunks of information. Thus, only the question was useful for finding the relevant chunks of information.

**Question:** Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?

**Answer (correct):** The Audi A4 2024 features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges.

**Status:** Hallucinations detected

**Detected hallucinations:** 'The Audi A4 2024' - Confidence = 0.9719486832618713

We retrieve the relevant documents for the query in order to understand this result:

In [66]:
test_retriever = retriever(corpus_dir = "./example_inputs/", debug = True)
docs = test_retriever.get_relevant_documents("Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?")
docs

Loading documents from ./example_inputs/...
Loading ford_f150_lightning_2024.txt...
Loading audi_a4_2024.txt...
Loading mercedes_cclass_2024.txt...
Loading tesla_model_s_2024.txt...
Loading toyota_camry_2024.txt...
Loading bmw_3series_2024.txt...
Loading jeep_wrangler_2024.txt...
Loading porsche_911_carrera_2024.txt...
Loading subaru_outback_2024.txt...
Loading honda_accord_2024.txt...
Filtering complex metadata...
Splitting 271 documents into chunks...
Initializing embeddings with model: text-embedding-3-small, deployment: text-embedding-3-small
Creating vector database with 271 documents...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Creating retriever from vector database...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


[Document(id='13defcb2-d1be-4552-a09e-334c08fb4e8d', metadata={'filename': 'bmw_3series_2024.txt', 'parent_id': '2259b4a98394c3141eb5a2b1cfd2d020', 'element_id': '13ef0961df231caad8d9f14677373220', 'source': './example_inputs/bmw_3series_2024.txt', 'category': 'NarrativeText', 'last_modified': '2025-07-24T15:17:12', 'file_directory': './example_inputs', 'filetype': 'text/plain'}, page_content='The available 12.3-inch digital instrument cluster provides customizable displays for vehicle information, navigation, and performance data. The head-up display projects critical information directly onto the windshield, allowing drivers to maintain focus on the road ahead.'),
 Document(id='fd9b56fd-be53-44ba-ad2e-2c7afc8f70e1', metadata={'source': './example_inputs/bmw_3series_2024.txt', 'last_modified': '2025-07-24T15:17:12', 'filetype': 'text/plain', 'element_id': '13ef0961df231caad8d9f14677373220', 'category': 'NarrativeText', 'file_directory': './example_inputs', 'filename': 'bmw_3series_202

**Conclusion**

LettuceDetect is a framework that detects hallucinations based on the abscence of data given a certain context that is assumed to be true, rather than on the actual validity of the evaluated answer.

Thus, when implemented in combination with a RAG system it should be used as a pre-processing step in order to point out possible conflicts with a given corpus of data, without assuming them to be necessarly true. In the previous example, the name of the car is correct. However, it is not shown in the relevant chunks (as these are about the requested information about the digital instrument display) and therefore is pointed out as an hallucination.

Later on, these conflicts can be evaluated by LLMs to determine their factual accuracy and provide an enhanced response.

## Test 2

The same test as before but now including the answer in the similarity search.

**Question:** Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?

**Answer (correct):** The Audi A4 2024 features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges.

**Status:** No hallucinations detected

**Detected hallucinations:**

We retrieve the relevant documents for the question and provided answer together. In this case, the relevant information relative to the "Audi A4 2024" model is included in the retrieved chunks of information.

In [67]:
test_retriever = retriever(corpus_dir = "./example_inputs/", debug = True)
docs = test_retriever.get_relevant_documents("Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?"
                                             +
                                             "The Audi 2024 features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges."
)
docs

Loading documents from ./example_inputs/...
Loading ford_f150_lightning_2024.txt...
Loading audi_a4_2024.txt...
Loading mercedes_cclass_2024.txt...
Loading tesla_model_s_2024.txt...
Loading toyota_camry_2024.txt...
Loading bmw_3series_2024.txt...
Loading jeep_wrangler_2024.txt...
Loading porsche_911_carrera_2024.txt...
Loading subaru_outback_2024.txt...
Loading honda_accord_2024.txt...
Filtering complex metadata...
Splitting 271 documents into chunks...
Initializing embeddings with model: text-embedding-3-small, deployment: text-embedding-3-small
Creating vector database with 271 documents...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Creating retriever from vector database...


[Document(id='b06c2a31-ca0c-43e4-877e-3f02cfbdd792', metadata={'file_directory': './example_inputs', 'parent_id': '97a4fe3f7cf75b893b0c058cd61d887a', 'category': 'NarrativeText', 'source': './example_inputs/audi_a4_2024.txt', 'element_id': 'b373aa446f6fd7009219d2ebf7ba0fe2', 'last_modified': '2025-07-24T15:17:12', 'filename': 'audi_a4_2024.txt', 'filetype': 'text/plain'}, page_content="Virtual Cockpit Technology The 2024 A4 features Audi's revolutionary Virtual Cockpit, a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges. This advanced system provides customizable layouts for vehicle information, navigation maps, entertainment controls, and performance data. Drivers can switch between Classic view that mimics traditional gauges and Infotainment view that prioritizes navigation and media information."),
 Document(id='608921cd-6d27-425a-b141-e1db919f82b6', metadata={'filetype': 'text/plain', 'element_id': 'b373aa446f6fd7009219d2ebf7ba0fe2', 'par

In the following code block, we join the retrievals for the question and answer separately in order to provide a greater context to the system.

In [68]:
test_retriever = retriever(corpus_dir = "./example_inputs/", debug = True)
docs_question = test_retriever.get_relevant_documents("Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?")
docs_answer = test_retriever.get_relevant_documents("The Audi 2024 features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges.")
docs_question + docs_answer

Loading documents from ./example_inputs/...
Loading ford_f150_lightning_2024.txt...


Loading audi_a4_2024.txt...
Loading mercedes_cclass_2024.txt...
Loading tesla_model_s_2024.txt...
Loading toyota_camry_2024.txt...
Loading bmw_3series_2024.txt...
Loading jeep_wrangler_2024.txt...
Loading porsche_911_carrera_2024.txt...
Loading subaru_outback_2024.txt...
Loading honda_accord_2024.txt...
Filtering complex metadata...
Splitting 271 documents into chunks...
Initializing embeddings with model: text-embedding-3-small, deployment: text-embedding-3-small
Creating vector database with 271 documents...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Creating retriever from vector database...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


[Document(id='eff043e4-a59f-41fe-8476-69a58853fcee', metadata={'parent_id': '2259b4a98394c3141eb5a2b1cfd2d020', 'filename': 'bmw_3series_2024.txt', 'filetype': 'text/plain', 'file_directory': './example_inputs', 'source': './example_inputs/bmw_3series_2024.txt', 'category': 'NarrativeText', 'element_id': '13ef0961df231caad8d9f14677373220', 'last_modified': '2025-07-24T15:17:12'}, page_content='The available 12.3-inch digital instrument cluster provides customizable displays for vehicle information, navigation, and performance data. The head-up display projects critical information directly onto the windshield, allowing drivers to maintain focus on the road ahead.'),
 Document(id='fd9b56fd-be53-44ba-ad2e-2c7afc8f70e1', metadata={'filetype': 'text/plain', 'source': './example_inputs/bmw_3series_2024.txt', 'parent_id': '2259b4a98394c3141eb5a2b1cfd2d020', 'file_directory': './example_inputs', 'last_modified': '2025-07-24T15:17:12', 'category': 'NarrativeText', 'element_id': '13ef0961df231c

**Conclusion**

We chose to retrieve documents for the question and answer separately, then combine them, to ensure both the context of the query and the specific entities or facts in the answer are represented. This prevents missing relevant information and reduces false positives in hallucination detection. Separate retrievals maintain clarity and provide LettuceDetect with a fuller context for accurate verification.

Additionally, providing more comprehensive context reduces the risk of false negatives, as it increases the likelihood that supporting evidence for true statements will be included in the verification process.


### Test 3

In this case, we successfully trick the model by providing a wrong response but combining actual information present in the retrieved documents.

**Question:** Which car features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?

**Answer (correct):** The Virtual Cockpit Technology features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges

**Status:** No hallucinations detected

**Detected hallucinations:**

### Test 4

In this test, we take advantage of the txt files being small in size so we can pass them directly to the system without splittig them into chunks, hoping that keeping all the information together will help in guiding the model.

In [69]:
def merge_documents(documents: list[Document]) -> Document:
    # Combine the content of all the documents into a single string
    combined_content = "\n\n".join(doc.page_content for doc in documents)

    # Combine the metadata of all documents into a single dictionary
    combined_metadata = {"sources": [doc.metadata for doc in documents]}

    # Create a new Document with the combined content and metadata
    return Document(page_content=combined_content, metadata=combined_metadata)

In [70]:
def load_full_documents(corpus_dir = "./corpus/", debug = False) -> list[str]:
    # Load all documents from the corpus directory
    loaded_docs = []
    if debug:
        print(f"Loading documents from {corpus_dir}...")
    for file in os.listdir(corpus_dir):
        # Load the file using UnstructuredLoader
        loader = UnstructuredLoader(corpus_dir + file, mode = 'single')
        docs = loader.load()

        # Join all the documents into a single one
        merged_docs = merge_documents(docs)

        # Append the document to the list of loaded documents
        loaded_docs.append(merged_docs)

    # Filter complex metadata from loaded documents
    if debug:
        print("Filtering complex metadata...")
    filtered_docs = filter_complex_metadata(loaded_docs)

    return filtered_docs

In [71]:
docs = load_full_documents("./example_inputs/", debug = True)
docs

Loading documents from ./example_inputs/...
Filtering complex metadata...


[Document(metadata={}, page_content="Ford F\n\n150 Lightning 2024\n\nElectric Truck Revolution\n\nThe 2024 Ford F-150 Lightning represents a revolutionary step in the evolution of America's best-selling truck, combining the capability and versatility that F-150 customers expect with advanced electric powertrain technology. This groundbreaking electric pickup maintains the rugged character and work-focused functionality of the traditional F-150 while delivering the benefits of electric propulsion, including instant torque, quiet operation, and advanced connectivity features.\n\nPowerful Electric Performance The 2024 F-150 Lightning features a sophisticated dual-motor electric powertrain that delivers exceptional performance across all driving conditions. The standard range model produces 452 horsepower and 775 lb-ft of torque, while the extended range configuration generates an impressive 580 horsepower and the same 775 lb-ft of instant torque. This immediate power delivery provides sup

In [72]:
def retriever_nosplit(corpus_dir = "./corpus/", debug = False):
    docs = load_full_documents(corpus_dir, debug = debug)
    vectordb = create_database(docs, debug = debug)
    if debug:
        print("Creating retriever from vector database...")
    retriever = vectordb.as_retriever()
    return retriever

In [73]:
test_retriever = retriever_nosplit(corpus_dir = "./example_inputs/", debug = True)
docs_question = test_retriever.get_relevant_documents("Which car model features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges?")
docs_answer = test_retriever.get_relevant_documents("The Audi 2024 features a 12.3-inch high-resolution digital instrument display that replaces traditional analog gauges.")
docs_question + docs_answer

Loading documents from ./example_inputs/...
Filtering complex metadata...
Initializing embeddings with model: text-embedding-3-small, deployment: text-embedding-3-small
Creating vector database with 10 documents...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


Creating retriever from vector database...


INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"
INFO: HTTP Request: POST https://paulo-mcw0r95x-eastus2.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-12-01-preview "HTTP/1.1 200 OK"


[Document(id='eff043e4-a59f-41fe-8476-69a58853fcee', metadata={'filetype': 'text/plain', 'filename': 'bmw_3series_2024.txt', 'element_id': '13ef0961df231caad8d9f14677373220', 'file_directory': './example_inputs', 'parent_id': '2259b4a98394c3141eb5a2b1cfd2d020', 'category': 'NarrativeText', 'last_modified': '2025-07-24T15:17:12', 'source': './example_inputs/bmw_3series_2024.txt'}, page_content='The available 12.3-inch digital instrument cluster provides customizable displays for vehicle information, navigation, and performance data. The head-up display projects critical information directly onto the windshield, allowing drivers to maintain focus on the road ahead.'),
 Document(id='fd9b56fd-be53-44ba-ad2e-2c7afc8f70e1', metadata={'parent_id': '2259b4a98394c3141eb5a2b1cfd2d020', 'category': 'NarrativeText', 'filetype': 'text/plain', 'file_directory': './example_inputs', 'source': './example_inputs/bmw_3series_2024.txt', 'filename': 'bmw_3series_2024.txt', 'last_modified': '2025-07-24T15:1